# Data Mining Versuch Music Clustering
* Autor: Prof. Dr. Johannes Maucher
* Datum: 16.10.2015

[Übersicht Ipython Notebooks im Data Mining Praktikum](Data Mining Praktikum.ipynb)

# Einführung
## Lernziele:
In diesem Versuch sollen Kenntnisse in folgenden Themen vermittelt werden:

* Zugriff auf Musikdateien
* Transcodierung von mp3 zu wav 
* Extraktion von Merkmalen in Musikdateien (Feature Extraction)
* Optimierung mit dem genetischen Algorithmus
* Selektion der aussagekräftigsten Merkmale (Feature Selection)
* Clustering von Musikfiles (automatische Playlistgenerierung)


## Vor dem Versuch zu klärende Fragen

### Transcodierung von MP3 nach WAV und Merkmalsextraktion
In diesem Versuch wird der MP3 Decoder [mpg123](http://www.mpg123.de/) eingesetzt. Installieren und testen sie diesen Decoder vor dem Versuch auf ihrem Rechner. Machen Sie sich zunächst mit dem in Kapitel [Gegebene Module zur Transcodierung und Feature Extraction](#Gegebene-Module-zur-Transcodierung-und-Feature-Extraction) aufgeführten Code vertraut. Versuchen Sie Funktion und Ablauf dieses Programms zu verstehen und beantworten Sie folgende Fragen.

1. Was versteht man unter den statistischen Größen _Mittelwert, Standardabweichung, Skewness und Kurtosis_?
2. Was beschreibt die Fourier-Transformierte eines zeitlich ausgedehnten Signals?
3. Mit welcher Samplingrate werden die WAV Dateien abgetastet?
4. Insgesamt werden 42 Merkmale pro Musiksequenz extrahiert. Beschreiben Sie kurz diese Merkmale



### Matching der Teilsequenzen

1. Nachdem für jedes Musikstück die beiden Teilsequenzen in Form der extrahierten Merkmale vorliegen: Wie kann die Ähnlichkeit zwischen Teilsequenzen ermittelt werden?
2. Welche Numpy- bzw. Scipy-Module können Sie für die Bestimmung der Ähnlichkeit zwischen Teilsequenzen einsetzen?

### Genetischer Algorithmus für die Merkmalsselektion

1. Beschreiben Sie die Prozesschritte im genetischen Algorithmus [Genetischer Algorithmus](https://www.hdm-stuttgart.de/~maucher/Python/FunktionenAlgorithmen/html/genAlgTSP.html)
2. In diesem Versuch wird davon ausgegangen, dass Merkmale dann gut sind, wenn durch sie die erste Teilsequenz eines Musikstücks durch einen ähnlichen Vektor wie die jeweils zweite Teilsequenz beschrieben wird. Wie kann mit dieser Annahme der genetische Algorithmus für die Merkmalsselektion angewandt werden. Unter Merkmalsselektion versteht man allgemein die Suche nach den $r$ besten Merkmalen aus einer Menge von insgesamt $R$ Merkmalen. In diesem Versuch werden initial $R=42$ Merkmale extrahiert, aus denen dann die besten $r<R$ Merkmale zu bestimmen sind. Überlegen Sie hierfür speziell wie die Fitnessfunktion, die Kreuzung und die Mutation zu realisieren sind.


### Clustering und Playlistgenerierung

1. Wie kann mit einem hierarchischen Clustering der Musikfiles eine Menge von Playlists erzeugt werden, so dass innerhalb einer Playlist möglichst ähnliche Titel zu finden sind?

# Durchführung
## Gegebene Module zur Transcodierung und Feature Extraction
Mit dem in diesem Abschnitt gegebenen Code werden die im Unterverzeichnis _BandCollection_ befindlichen mp3-Files zunächst in wave decodiert. Danach werden aus den wave Dateien Audiomerkmale erhoben.

Von jedem Musikstück werden zwei disjunkte Teilsequenzen erhoben und von beiden Teilsequenzen jeweils ein Merkmalsvektor gebildet. Der Grund hierfür ist: Für die später folgende Bestimmung der wichtigsten Merkmale (Merkmalsselektion mit dem genetischen Algorithmus), wird angenommen dass Merkmale dann gut sind, wenn die aus ihnen gebildeten Merkmalsvektoren für Teilsequenzen des gleichen Musikstücks nahe beieinander liegen und die Merkmalsvektoren von Teilsequenzen unterschiedlicher Musikstücke weiter voneinander entfernt sind. In der Merkmalsselektion werden dann die Merkmale als relevant erachtet, für die diese Annahme zutrifft. 

**Aufgaben:**

1. Stellen Sie im unten gegebenen Code die Verzeichnisse für Ihre Musikdateien (aktuell Unterverzeichnis _BandCollection_) und für den Ort Ihres _mpg123_ Decoders richtig ein.
2. Die verwendete Musiksammlung sollte mindestens 5 verschiedene Interpreten möglichst unterschiedlicher Genres enthalten. Von jedem Interpret sollten mehrere Titel (evtl. ein ganzes Album) enthalten sein.
3. Führen Sie den in diesem Abschnitt gegebenen Programmcode zur Audiofeature-Extraction aus. Damit werden für alle Musiksequenzen jeweils 42 Merkmale extrahiert. Die extrahierten Merkmalsvektoren der jeweils ersten Sequenz werden in das File _FeatureFileTrainingAllList1.csv_ geschrieben, die der zweiten Teilsequen in das File _FeatureFileTestAllList2.csv_. 


In [ ]:
import subprocess
import wave
import struct
import numpy
import os
import pandas as pd

numpy.set_printoptions(precision=2,suppress=True)

#Names of features extracted in this module
FeatNames=["amp1mean","amp1std","amp1skew","amp1kurt","amp1dmean","amp1dstd","amp1dskew","amp1dkurt","amp10mean","amp10std",
           "amp10skew","amp10kurt","amp10dmean","amp10dstd","amp10dskew","amp10dkurt","amp100mean","amp100std","amp100skew",
           "amp100kurt","amp100dmean","amp100dstd","amp100dskew","amp100dkurt","amp1000mean","amp1000std","amp1000skew",
           "amp1000kurt","amp1000dmean","amp1000dstd","amp1000dskew","amp1000dkurt","power1","power2","power3","power4",
           "power5","power6","power7","power8","power9","power10"]

In [ ]:
def moments(x):
    mean = x.mean()
    std = x.var()**0.5
    skewness = ((x - mean)**3).mean() / std**3
    kurtosis = ((x - mean)**4).mean() / std**4
    return [mean, std, skewness, kurtosis]

In [ ]:
#Feature category 2: Frequency domain parameters
def fftfeatures(wavdata):
    f = numpy.fft.fft(wavdata)
    f = f[2:(int(f.size / 2) + 1)]
    f = abs(f)
    total_power = f.sum()
    f = numpy.array_split(f, 10)
    return [e.sum() / total_power for e in f]

In [ ]:
#Creating the entire feature vector per music-file
def features(x):
    x = numpy.array(x)
    f = []

    xs = x
    diff = xs[1:] - xs[:-1]
    f.extend(moments(xs))
    f.extend(moments(diff))

    xs = x.reshape(-1, 10).mean(1)
    diff = xs[1:] - xs[:-1]
    f.extend(moments(xs))
    f.extend(moments(diff))

    xs = x.reshape(-1, 100).mean(1)
    diff = xs[1:] - xs[:-1]
    f.extend(moments(xs))
    f.extend(moments(diff))

    xs = x.reshape(-1, 1000).mean(1)
    diff = xs[1:] - xs[:-1]
    f.extend(moments(xs))
    f.extend(moments(diff))

    f.extend(fftfeatures(x))
    return f

In [ ]:
def read_wav(wav_file):
    """Returns two chunks of sound data from wave file."""
    w = wave.open(wav_file)
    n = 60 * 10000
    if w.getnframes() < n * 3:
        raise ValueError('Wave file too short')
    #For each music file 2 sequences, each containing n frames are subtracted. The first sequence starts at postion n,
    #the second sequence starts at postion 2n. The reason for extracting 2 subsequences is, that later on we like to
    #find the best features and in this exercise we assume that good features have the property that they are similar for 2 subsequences
    #of the same song, but differ for subsequences of different songs.
    w.setpos(n)
    frames = w.readframes(n)
    wav_data1 = struct.unpack('%dh' % n, frames)
    frames = w.readframes(n)
    wav_data2 = struct.unpack('%dh' % n, frames)
    return wav_data1, wav_data2

In [ ]:
def compute_chunk_features(mp3_file):
    """Return feature vectors for two chunks of an MP3 file."""
    # Extract MP3 file to a mono, 10kHz WAV file
    #mpg123_command = 'C:\Program Files (x86)\mpg123-1.22.0-x86\mpg123-1.22.0-x86\\mpg123.exe -w "%s" -r 10000 -m "%s"'
    #mpg123_command = 'C:\\Program Files (x86)\\mpg123-1.21.0-x86-64\\mpg123.exe -w "%s" -r 10000 -m "%s"'
    mpg123_command = 'C:\mpg123\\mpg123.exe -w "%s" -r 10000 -m "%s"'
    out_file = 'temp.wav'
    cmd = mpg123_command % (out_file, mp3_file)
    temp = subprocess.call(cmd)
    # Read in chunks of data from WAV file
    wav_data1, wav_data2 = read_wav(out_file)
    # We'll cover how the features are computed in the next section!
    return numpy.array(features(wav_data1)), numpy.array(features(wav_data2))

In [ ]:
fileList=[]
featureList1=[]
featureList2=[]
#Specify the name of the directory, which contains your MP3 files here.
# This directory should contain for each band/author one subdirectory, which contains all songs of this author
for path, dirs, files in os.walk('./BandCollection'):
    #print '-'*10,dirs,files
    for f in files:
        if not f.endswith('.mp3'):
            # Skip any non-MP3 files
            continue
        mp3_file = os.path.join(path, f)
        print(mp3_file)
        # Extract the track name (i.e. the file name) plus the names
        # of the two preceding directories. This will be useful
        # later for plotting.
        tail, track = os.path.split(mp3_file)
        tail, dir1 = os.path.split(tail)
        tail, dir2 = os.path.split(tail)
        # Compute features. feature_vec1 and feature_vec2 are lists of floating
        # point numbers representing the statistical features we have extracted
        # from the raw sound data.
        try:
            feature_vec1, feature_vec2 = compute_chunk_features(mp3_file)
        except:
            print("Error: Chunk Features failed")
            continue
        #title=str(track)
        title=str(dir1)+'\\'+str(track)
        print('-'*20+ title +'-'*20)
        #print "       feature vector 1:",feature_vec1
        #print "       feature vector 2:",feature_vec2
        fileList.append(title)
        featureList1.append(feature_vec1)
        featureList2.append(feature_vec2)

# Write feature vecotrs of all music files to pandas data-frame
MusicFeaturesTrain=pd.DataFrame(index=fileList,data=numpy.array(featureList1),columns=FeatNames)
MusicFeaturesTrain.to_csv("FeatureFileTrainingAllList1.csv")

MusicFeaturesTest=pd.DataFrame(index=fileList,data=numpy.array(featureList2),columns=FeatNames)
MusicFeaturesTest.to_csv("FeatureFileTestAllList2.csv")

In [ ]:
import platform
 
print(platform.python_version())

## Matching der Teilsequenzen
In diesem Abschnitt soll ein Verfahren implementiert werden, mit dem die Übereinstimmung der ersten Teilsequenz eines Musikstücks mit den zweiten Teilsequenzen aller anderen Musikstücke berechnet werden kann.

**Aufagben:**
1. Lesen Sie die im vorigen Teilversuch angelegten zwei csv-Dateien in jeweils einen eigenen Pandas Dataframe ein.
2. Skalieren Sie beide Teilsequenzmengen, so dass alle Merkmale eine Standardabweichung von 1 aufweisen. Z.B. mit [http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html).
2. Bestimmen Sie zu jeder Teilsequenz aus der Datei _FeatureFileTrainingAllList1.csv_ die euklidische Distanz zu allen Teilsequenzen aus der Datei _FeatureFileTestAllList2.csv_ und schreiben Sie diese Distanzen in eine aufsteigend geordnete Liste. Schreiben Sie auch die zugehörigen Argumente (Teilsequenzen) in eine geordnete Liste, sodass für jede Teilsequenz aus _FeatureFileTrainingAllList1.csv_ die am nächsten liegende Teilsequenz aus _FeatureFileTestAllList2.csv_ an erster Stelle steht, die zweitnächste Teilsequenz an zweiter usw.
3. Bestimmen Sie über alle Teilsequenzen aus _FeatureFileTrainingAllList1.csv_ den **mittleren Rang** an dem die zugehörige zweite Teilsequenz erscheint. Liegt z.B. für die erste Teilsequenz des Musikstücks A die zweite Teilsequenz nur an fünfter Stelle der geordneten nächsten Nachbarliste. Dann würde diese Teilsequenz mit dem Rang 5 in den Mittelwert einfließen.
4. Bestimmen Sie jetzt den mittleren Rang, für den Fall, dass _correlation_ anstelle _euclidean_ als Ähnlichkeitsmaß verwendet wird. Welches Ähnlichkeitsmaß ist für diese Anwendung zu bevorzugen?
5. Diskutieren Sie das Ergebnis


In [1]:
import pandas as pd
df1 = pd.DataFrame.from_csv("FeatureFileTrainingAllList1.csv")
df2 = pd.DataFrame.from_csv("FeatureFileTestAllList2.csv")

import sklearn.preprocessing as skp
import numpy as np

for df in [df1,df2]:
    for i in range(0,len(df.columns)):
        df.iloc[:,i] = skp.scale(df.iloc[:,i])
        
df1

,amp1mean,amp1std,amp1skew,amp1kurt,amp1dmean,amp1dstd,amp1dskew,amp1dkurt,amp10mean,amp10std,...,power1,power2,power3,power4,power5,power6,power7,power8,power9,power10
Adele\01 Hometown Glory.mp3,0.232577,-0.797468,2.872964,0.355331,0.068897,-0.627371,2.381091,1.687820,0.232577,-1.095311,...,-1.391123,1.460863,2.699313,1.737095,-0.697896,-0.844317,-1.586884,-1.366235,-0.183381,-1.400626
Adele\02 I'll Be Waiting.mp3,0.180263,-0.173128,0.582953,-0.857496,0.298870,0.715917,0.757480,-0.347053,0.180263,-0.372479,...,-0.617365,0.076090,-0.130870,1.228305,1.015333,0.672293,-0.294418,-0.090637,-0.180477,-0.854365
Adele\03 Don't You Remember.mp3,0.146118,-0.192314,1.083192,-0.561409,0.331443,0.864760,0.901385,-0.326521,0.146118,-0.474315,...,-1.288164,0.436651,0.091413,1.389280,0.054543,0.478940,-0.507203,1.210078,1.508138,-1.493994
Adele\04 Turning Tables.mp3,0.246328,-0.738500,1.522685,-0.651896,-0.083175,0.213988,1.305449,0.555752,0.246328,-1.086144,...,-2.022388,0.742014,0.894909,1.673301,0.153279,0.738740,-0.551655,-0.110777,1.768929,0.295994
Adele\05 Set Fire To The Rain.mp3,0.206761,-0.324045,0.949064,-0.777781,0.342529,0.363965,1.195427,-0.453736,0.206761,-0.547233,...,-1.075119,0.647763,0.001291,1.905737,0.323134,0.366649,-0.711752,-0.002929,0.410124,-0.973227
Adele\06 If It Hadn't Been For Love.mp3,0.280348,-0.478968,0.995088,-0.575096,1.016455,0.771114,1.086029,-0.032049,0.280348,-0.830216,...,-1.412519,0.346507,2.001958,1.103096,0.131258,0.354669,-0.708603,-0.345480,0.460725,-0.537021
Adele\08 Take It All.mp3,0.247481,-0.926990,2.954399,0.099251,-0.574909,0.359009,1.076136,1.726050,0.247481,-1.259589,...,-2.074538,0.808417,-0.659491,1.096841,3.373815,1.770101,0.014934,0.035694,0.509044,-0.974029
Adele\09 Rumour Has It.mp3,0.246957,-0.332796,0.273734,-0.740430,0.234018,-0.544921,0.683626,-0.016039,0.246957,-0.262916,...,0.653108,-0.444331,0.542047,0.290555,-0.444201,-0.459225,-1.150753,-0.349448,-0.055841,0.108565
Adele\11 One And Only.mp3,0.155441,-0.180227,0.548754,-0.726173,0.082731,0.426878,1.368922,0.375488,0.155441,-0.254850,...,-0.815554,-0.792671,1.157053,2.286713,0.533827,0.366551,-0.363925,-0.439889,-0.045196,0.270724
Adele\12 Lovesong.mp3,0.258596,-0.672856,1.435505,-0.556085,-0.283715,-1.583634,2.012882,1.850948,0.258596,-0.459928,...,1.429404,0.446433,-0.420072,-0.530622,-1.565266,-0.643870,-1.503402,-1.404455,1.255008,-0.729013


In [2]:
for df in [df1,df2]:
    print("________________________________")
    for i in range(0,len(df.columns)):
        print(np.std(df.iloc[:,i]))

________________________________
0.9999999999999996
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999998
0.9999999999999999
0.9999999999999998
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
0.9999999999999998
1.0
1.0000000000000002
0.9999999999999999
0.9999999999999997
1.0
________________________________
0.9999999999999999
1.0
1.0
0.9999999999999998
1.0
1.0000000000000002
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0000000000000002
0.9999999999999999
1.0
0.9999999999999999
1.0000000000000002
1.0
0.9999999999999999
0.9999999999999998
0.9999999999999998
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
1.0000000000000002
1.0000000000000002
0.9999999999999998
1.0
1.0
0.9999999999999998
1.0000000000000002
1.0
0.99999999999999

In [3]:
import operator

euclid_list = {}
for index1, row1 in df1.iterrows():
    inner_dic = {}
    for index2, row2 in df2.iterrows():
        value = np.linalg.norm((np.transpose(row1.tolist()))     -     (np.transpose(row2.tolist())))
        inner_dic[index2] = value
    sorted_list = sorted(inner_dic.items(), key=operator.itemgetter(1))    
    euclid_list[index1] = sorted_list
    #euclid_list[index1] = inner_dic

In [ ]:
euclid_list

In [ ]:
ranks = []
for song in euclid_list:
    for rank in range(0,len(euclid_list[song])):
        if(euclid_list[song][rank][0] == song):
            ranks.append((song,rank))
ranks

In [ ]:
summ = 0
for song in ranks:
    summ += song[1]
print(summ)
result = summ/len(ranks)
print(result)

In [ ]:
import operator
import numpy as np
from scipy.stats import pearsonr

pearson_list = {}
for index1, row1 in df1.iterrows():
    inner_dic = {}
    for index2, row2 in df2.iterrows():
        r,p = pearsonr((np.transpose(row1.tolist()))     ,     (np.transpose(row2.tolist())))
        if(r<0):
            value = 1+r
        else:
            value = 1-r
        inner_dic[index2] = value
    sorted_list = sorted(inner_dic.items(), key=operator.itemgetter(1))    
    pearson_list[index1] = sorted_list

In [ ]:
pearson_list

In [ ]:
ranks = []
for song in pearson_list:
    for rank in range(0,len(pearson_list[song])):
        if(pearson_list[song][rank][0] == song):
            ranks.append((song,rank))
ranks

In [ ]:
summ = 0
for song in ranks:
    summ += song[1]
print(summ)
result = summ/len(ranks)
print(result)

## Merkmalsauswahl mit dem genetischen Algorithmus
In diesem Abschnitt soll unter Anwendung eines selbst zu implementierenden genetischen Algorithmus eine Untermenge wichtiger Merkmale aus den insgesamt 42 angelegten Merkmalen berechnet werden.
Als Vorlage kann hierfür die Implementierung für die [Lösung des TSP Problems](https://www.hdm-stuttgart.de/~maucher/Python/FunktionenAlgorithmen/html/genAlgTSP.html) herangezogen werden. Anzupassen sind dann jedoch mindestens die Fitness-Funktion, die Kreuzungs- und die Mutationsfunktion. Die Fitness soll so wie im vorigen Teilabschnitt mit dem mittleren Rang berechnet werden. Die Populationsgröße, die Anzahl der auszuwählenden Merkmale und die Anzahl der Iterationen sollen als Parameter einstellbar sein.

Der Fitnesswert des besten Individuums in der Population soll in jeder Iteration gespeichert werden. Der Verlauf dieses besten Fitness-Wertes über den Fortlauf der Iterationen soll graphisch ausgegeben werden.

Ein Pandas Frame, der nur die berechneten wichtigsten Merkmale aus _FeatureFileTrainingAllList1.csv_ enthält soll angelegt und in die csv Datei _subFeaturesTrain1.csv_ geschrieben werden.

**Aufgaben:**
1. Implementieren Sie die die Merkmalsauswahl mit dem genetischen Algorithmus entsprechend der o.g. Beschreibung
2. Beschreiben Sie kurz das Konzept ihrer Kreuzungs- und Mutationsfunktion. 
3. Bestimmen Sie eine möglichst kleine Merkmalsuntermenge mit einem möglichst guten mittleren Rang? Geben Sie sowohl die gefundenen wichtigsten Merkmale als auch den zugehörigen mittleren Rang an.
4. Um wieviel verschlechtert sich der Mittlere Rang, wenn nur die 10 wichtigsten Merkmale benutzt werden?

#### Genetischer Algorithmus für die Music Feature Selection

In [ ]:
popsizeMax = 100
attributes = 20
iterations = 10



In [ ]:
import copy

class Individual:
    'Common base class for all employees'

    def __init__(self, nofattributes,used_attributes,attribute_settings=0,random=True,similarity="euclid"):
        self.nofattributes = nofattributes
        self.used_attributes = used_attributes
        if(random):
            settings = np.zeros(nofattributes)
            for i in range(0,used_attributes):
                success = False
                while(not success):
                    index = np.random.randint(0,self.nofattributes)
                    if(settings[index]==0):
                        settings[index] = 1
                        success=True
            self.attribute_settings = [*settings]
        else:
            self.attribute_settings = attribute_settings
        self.calcFitness(similarity)
   
    def repair(self):
        while(sum(self.attribute_settings)>self.used_attributes):
            print("REPAIR: deleting random setting")
            self.attribute_settings[np.random.randint(0,self.nofattributes)] = 0
            
        while(sum(self.attribute_settings)<self.used_attributes):
            print("REPAIR: adding random setting")
            self.attribute_settings[np.random.randint(0,self.nofattributes)] = 1
        
        return self
    
    def cross(self,partner,mutprob,similarity="euclid"):
        #print("entering cross")
        found_segment = False
        itera = 0
        locus_length = 0
        while(not found_segment):
            itera+=1
            locus_start = np.random.randint(0,self.nofattributes-1)
            locus_end = np.random.randint(locus_start+1,self.nofattributes)
            
            locus_length = locus_end-locus_start
            
            sum_self = sum(self.attribute_settings[locus_start:locus_end])
            sum_partner = sum(partner.attribute_settings[locus_start:locus_end])
            
            if(sum_self == sum_partner and locus_length> int(self.nofattributes/10)):
                found_segment=True
                #print("needed " +str(itera)+ "tries for locus")
        
        #print("locus start: "+str(locus_start)+" locus end: "+str(locus_end))
        #print("locus length was: "+str(locus_length))
        #print("found segments:")
        #print(self.attribute_settings[locus_start:locus_end])
        #print(partner.attribute_settings[locus_start:locus_end])
        
        settings = copy.deepcopy(self.attribute_settings)
        settings[locus_start:locus_end] = partner.attribute_settings[locus_start:locus_end]
        
        child = Individual(self.nofattributes,self.used_attributes,settings,random=False)
        child.mutate(mutprob)
        #print("calculating child fitness...")
        child.calcFitness(similarity)
        #print("new child fitness is:"+str(child.fitness))
        return child
           
    def mutate(self,probability):       
        for i in range(0,self.nofattributes):
            if(np.random.rand()<probability):
                if(self.attribute_settings[i]==1):
                    self.attribute_settings[i] = 0
                else:
                    self.attribute_settings[i] = 1
        
        self.repair()
        return self
        
    def calcFitness(self,similarity):
        import operator
        sim_list = {}
        for index1, row1 in df1.iterrows():
                inner_dic = {}
                for index2, row2 in df2.iterrows():
                    
                    # only used_attributes
                    r1 = row1.tolist()
                    r2 = row2.tolist()
                    
                    rn1 = []
                    rn2 = []
                    for i in range(0,len(self.attribute_settings)):
                        if(self.attribute_settings[i]==1):
                            rn1.append(r1[i])
                            rn2.append(r2[i])
                     
                    if(similarity=="euclid"):
                        value = np.linalg.norm(np.transpose(rn1)-np.transpose(rn2))
                    elif(similarity=="pearson"):
                        r,p = pearsonr((np.transpose(rn1))     ,     (np.transpose(rn2)))
                        if(r<0):
                            value = 1+r
                        else:
                            value = 1-r
                    else:
                        print("WRONG SIMILARITY OPTION")
                
                    inner_dic[index2] = value
                sorted_list = sorted(inner_dic.items(), key=operator.itemgetter(1))    
                sim_list[index1] = sorted_list
                
                
        ranks = []
        for song in sim_list:
            for rank in range(0,len(sim_list[song])):
                if(sim_list[song][rank][0] == song):
                    ranks.append((song,rank))
                
        summ = 0
        for song in ranks:
            summ += song[1]
        self.fitness = summ/len(ranks)
        return self.fitness

In [ ]:
for i in range(0,100):
    a = Individual(42,20)
    b = Individual(42,20)
    c = a.cross(b,0)

In [2]:
from GA import *

In [3]:
for i in  range(0,3):
    a = Individual(42,20,df1,df2)
    print(a.calcFitness("euclid",df1,df2))

2.35
3.6333333333333333
4.383333333333334


In [ ]:
for i in range(0,3):
    b = Individual(42,5)
    print(b.calcFitness("pearson"))

In [ ]:
for i in range(0,3):
    c = Individual(42,42)
    print(c.calcFitness("euclid"))

In [ ]:
for i in range(0,3):
    c = Individual(42,42)
    print(c.calcFitness("pearson"))

In [ ]:
c = a.cross(b,0.2)

In [ ]:
c.attribute_settings

In [ ]:
start --> anzahl merkmale, populationsgröße, iterationen

In [ ]:
#Your Code
class Population:
    'Common base class for all employees'
    def __init__(self,nofattributes,used_attributes,popsize,population=[],mutprob=0.01):
        self.popsize = popsize
        self.population = population
        self.nofattributes = nofattributes
        self.used_attributes = used_attributes
        self.mutprob = mutprob

    def add(self,ind):
        self.population.append(ind)
    
    def random_populate(self):
        self.population = []
        while(len(self.population)<self.popsize):
            self.add(Individual(self.nofattributes,self.used_attributes))
            
     
    def getMeanFitness(self):
        summ = 0
        for ind in self.population:
            summ+=ind.fitness
        return summ/len(self.population)
    
    def getMaxFitness(self):
        best = self.population[0].fitness
        best_ind = self.population[0]
        for ind in self.population:
            if(ind.fitness<best):
                best = ind.fitness
                best_ind = ind 
        return(best_ind)
    
    def selectPartners(self):
        partners = []
        
        dic = {}
        for ind in self.population:
            dic[ind] = (1/ind.fitness)
            
        max = sum(dic.values())
        
        for i in range(0,2):
            pick = np.random.uniform(0,max)
            current = 0
            for key, value in dic.items():
                current += value
                if current > pick:
                    partners.append(key)
                    break
                    
        return partners
    
    def getNextGeneration(self):
        new_pop = Population(self.nofattributes,self.used_attributes,self.popsize,population=[])

        #def(worker --> (new_pop))
        
        #print("len(new_pop.population)="+str(len(new_pop.population)))
        #print("new_pop.popsize="+str(new_pop.popsize))
        while(len(new_pop.population) < new_pop.popsize):
            #print("entering while loop - selecting partners...")
            partners = self.selectPartners()
            
            from multiprocessing import Process, Queue
            import g

            if __name__ == '__main__':    
                # Define an output queue
                output=Queue()

                # Setup a list of processes that we want to run
                p = Process(target=g.g, args=(partners[0],partners[1],self.mutprob,output))

                # Run process
                p.start()

                # Exit the completed process
                p.join()

                # Get process results from the output queue
                result = output.get(p)

                new_pop.add(result) 
                #print(result)
                
            #2 partners passed
            
            #child = partners[0].cross(partners[1],self.mutprob)
            #print("created child: "+str(child))
            #new_pop.add(child)
        
        return new_pop

In [4]:
p = Population(42,20,5)

In [5]:
p.random_populate()

TypeError: __init__() missing 2 required positional arguments: 'df1' and 'df2'

In [ ]:
c = p.getNextGeneration()

In [ ]:
print(p.population)

In [ ]:
p.getMeanFitness()

In [ ]:
a  =p.selectPartners()

In [ ]:
a[0]

In [ ]:
p.population

In [ ]:
### CREATION
# first population random
p = Population(42,20,25)
p.random_populate()

iteration = 0
max_iterations = 30

iter_best = []

while(p.getMeanFitness()>0.5 and iteration<max_iterations):
    print("iteration"+str(iteration))
    #print("population mean fitness:"+str(p.getMeanFitness()))
    best = p.getMaxFitness()
    print("population max fitness individual: "+str(best)+"with: "+str(best.fitness))
    iter_best.append((iteration,best,best.fitness))
    iteration+=1
    
    #print("pop length:"+str(len(p.population)))
    p = p.getNextGeneration()
    #print(p)
    #print(p.population)

In [ ]:
iter_best

In [ ]:
for i in p.selectPartners():
    print(1/p.selectPartners()[i])

In [ ]:
from multiprocessing import Process, Queue

#Having the function definition here results in
#AttributeError: Can't get attribute 'f' on <module '__main__' (built-in)>

#The solution seems to be importing the function from a separate file.

import f

#Also, the original version of f only had a print statement in it.  
#That doesn't work with Process - in the sense that it prints to the console 
#instead of the notebook.
#The trick is to let f write the string to print into an output-queue.
#When Process is done, the result is retrieved from the queue and printed.

if __name__ == '__main__':    

   # Define an output queue
   output=Queue()

   # Setup a list of processes that we want to run
   p = Process(target=f.f, args=('Bob',output))

   # Run process
   p.start()

   # Exit the completed process
   p.join()

   # Get process results from the output queue
   result = output.get(p)

   print(result)



In [ ]:
multi()

In [ ]:
if __name__ == '__main__':
    jobs = []
    for i in range(5):
        p = multiprocessing.Process(target=worker)
        jobs.append(p)
        p.start()

In [ ]:
worker()

#### Music Feature Selection

In [ ]:
#Your Code

## Clustering und automatische Playlistgenerierung
Implementieren Sie ein hierarchisches Clustering aller Subsequenzen in _subFeaturesTrain1.csv_. Diese _.csv_-Datei enthält nur die im vorigen Schritt ermittelten wichtigsten Merkmale. Das hierarchische Clustering ist in einem Dendrogram der Art wie in der unten gegebenen Abbildung zu visualisieren.

Die gefundenen Cluster sind mit den zugehörigen Musiktiteln in der Konsole auszugeben. 

**Aufgaben:**

1. Optimieren Sie die Parameter

    1. metric (Ähnlichkeitsmaß)
    2. linkage method
    3. Clusteranzahl
    
2. Für welche Parameterkonstellation erlangen Sie das für Sie subjektiv betrachtet günstigste Ergebnis?
3. Überlegen Sie sich Ansätze um diese Art der Musikgruppierung zu verbessern?

![Abbildung Music Clustering](https://www.hdm-stuttgart.de/~maucher/ipnotebooks/DataMining//Bilder/playlistCluster.png "Music Clustering")

## BACKUP CODE

In [ ]:
def createNextGeneration(population):
    new_pop = Population(population.nofattributes,population.used_attributes,population.popsize)
    while(len(new_pop.population) < new_pop.popsize):
        partners = population.selectPartners()
        new_pop.add(partners[0].cross(partners[1],self.mutprob))    
    return new_pop